<a href="https://colab.research.google.com/github/jjeongab/property_management/blob/main/%EB%A7%A4%EB%AC%BC%EA%B4%80%EB%A6%AC%EC%B6%94%EC%B2%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img align="left" style=" width:40px;  " > Content-Based Filtering on 매물관리

In this exercise, you will implement content-based filtering using a neural network to build a recommender system for movies.


In [ ]:
!pip install pymysql

In [ ]:
import pandas as pd
import pymysql
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
sns.set()
import re
import os
import random
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
#matplotlib한글깨짐 방지
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

fc-cache: succeeded


In [ ]:
plt.rc('font', family='NanumBarunGothic')

###Data crawling for trade area analysis

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dfs = dict() #dictionary where key corresponds to region and item responds to the data accordingly to that region

In [ ]:
directory = '/content/drive/MyDrive/상가(상권)정보_20230630'

In [ ]:
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if filename.endswith('.txt'):
      continue
    else:
      filename = filename.split('_')
      filename = filename[1].split('.')

      dfs[filename[0]] = pd.read_csv(f)

In [ ]:
#confirmed to allocate each region to a dataFrame having correspding region's data
for region, data in dfs.items():
  print(region)
  print(type(data))

In [ ]:
df_seoul = dfs['서울']
df_seoul.columns

In [ ]:
variables = ['상가업소번호', '상호명',  '위도', '경도', '표준산업분류명', '신우편번호', '층정보', '상권업종소분류명', '도로명주소'] #use the 상권업중소분류명 as target variables

In [ ]:
df_seoul = df_seoul[variables]
df_seoul
df_seoul.index

RangeIndex(start=0, stop=473762, step=1)

###Feature engineering
For null values in floor information, just replace the null value as 0 and also try to see the number of stores nearby the selected building to see any business with highest frequencies. Also consider 상권업종소분류명 as target variable

In [ ]:
#replace the value NaN value by 1
df_seoul['층정보'].fillna(1, inplace = True)
df_seoul

In [ ]:
df_seoul.sort_values(['경도', '위도'], ascending = True, inplace = True)
df_seoul = df_seoul.reset_index(drop=True)
df_seoul

In [ ]:
df_seoul.iloc[0]

In [ ]:
#trying to allocate unique business types of that place to df_biz dictionary
df_seoul_biz = df_seoul['상권업종소분류명'].unique() #returns list that contains all elements of 상권업종소분류명

In [ ]:
biz_type = len(df_seoul_biz)
print(biz_type)

246


In [ ]:
df_seoul

In [ ]:
#first of all since there are many corresponding data regarding seoul,lets dive into seoul data first
#lets get the data that has 도로명주소(since it implies nearby region) and also the building itself
 #should i add on random data that shows the ave review and review count
rows = df_seoul.shape[0]
def create_random_dataframe_with_float_numbers(min_value, max_value, number_of_rows):
  """Creates a Pandas DataFrame that has random float numbers within the range."""
  random_floats = ["{0:.2f}".format(random.uniform(min_value, max_value)) for i in range(number_of_rows)]
  dataframe = pd.DataFrame({
    "random_float": random_floats
  })
  return dataframe
df_seoul['review'] = create_random_dataframe_with_float_numbers(1, 5, rows)
df_seoul['ave_count'] = np.random.randint(1, 10000, (rows, 1))
df_seoul

In [ ]:
biz_type = df_seoul['상권업종소분류명'].unique()

In [ ]:
"""biz_label = dict()
for index, biz in enumerate(biz_type):
  biz_label[index] = biz
biz_label
"""


In [ ]:
from sklearn.preprocessing import LabelEncoder
from numpy import dot
from numpy.linalg import norm

In [ ]:
#use labelencoder to apply hashing numbers on each
le = LabelEncoder()
df_seoul['상권업종소분류명'] = le.fit_transform(df_seoul['상권업종소분류명'])

In [ ]:
df_seoul

In [ ]:
#checking how the label encoder is enco
#classes = le.classes_
#print(classes)de
#some vectors neeed
def decoding(number): #returnes the decoded value of business model when given the encoded number in this function
  original_value = le.inverse_transform([number])
  return original_value
def normalize(data):
    '''
    Gets dataframe and normalizes input data to be between 0 and 1
    '''
    min_val = min(data)
    if min_val < 0:
        data = [x + abs(min_val) for x in data]
    max_val = max(data)
    return [x/max_val for x in data]
def cosine_sim(v1,v2):
    '''
    Calculates the cosine similarity between two vectors
    '''
    return dot(v1,v2)/(norm(v1)*norm(v2))
def ohe(df, enc_col):
    '''
    One hot encodes specified columns and adds them back
    onto the input dataframe
    '''

    ohe_df = pd.get_dummies(df[enc_col])
    ohe_df.reset_index(drop = True, inplace = True)
    return pd.concat([df, ohe_df], axis = 1)

In [ ]:
businesses = df_seoul[['위도', '경도', '상권업종소분류명']] #corresponds to name, business_model, 'description'
cosine_similarities = businesses.dot(businesses.T) / (businesses.norm(axis=1) * businesses.norm(axis=1).T)